In [ ]:
import os
import pyvista as pv

pv.set_jupyter_backend('static')
os.environ['DISPLAY'] = ':99.0'
os.environ['PYVISTA_OFF_SCREEM'] = 'true'

# Data loading

## 3dMD

In [ ]:
import pyvista as pv

mesh_path = '/mnt/d/knpob/4-data/20211229-DynaBreast4D/3dmd/0kmh_8marker_bra/meshes/static_standing_with_bra.000001.obj'
mesh = pv.read(mesh_path)
texture = pv.read_texture(mesh_path.replace('.obj', '.jpg'))

In [ ]:
import numpy as np

points_3dmd = np.load('output/points_3dmd.npy')
points_3dmd

## Vicon

In [ ]:
import pandas as pd

vicon_path = '/mnt/d/knpob/4-data/20211229-DynaBreast4D/vicon/0kmh_8marker_bra/vicon.csv'
df = pd.read_csv(vicon_path, skiprows=4).iloc[:, 2:].dropna(axis=1)
df

In [ ]:
frame = 0
points_vicon = df.values[frame].reshape(-1, 3)
points_vicon

## Visualization

In [ ]:
scene = pv.Plotter()
scene.add_points(points_3dmd, point_size=10, color='yellow', render_points_as_spheres=True)
scene.add_points(points_vicon, point_size=10, color='green', render_points_as_spheres=True)
scene.add_mesh(mesh, texture=texture)
scene.camera_position = 'xy'
scene.show()

# Registration

## Pre-alignment

In [ ]:
from mesh4d import obj3d

pcd_vicon = obj3d.np2pcd(points_vicon)
r_pre = pcd_vicon.get_rotation_matrix_from_xyz((-np.pi/2, 0, 0))
pcd_vicon.rotate(r_pre, center=(0, 0, 0))
points_vicon_pre = obj3d.pcd2np(pcd_vicon)

In [ ]:
scene = pv.Plotter()
scene.add_points(points_3dmd, point_size=10, color='yellow', render_points_as_spheres=True)
scene.add_points(points_vicon_pre, point_size=10, color='green', render_points_as_spheres=True)
scene.add_mesh(mesh, texture=texture)
scene.camera_position = 'xy'
scene.show()

## Refined registration with CPD

In [ ]:
from probreg import cpd
tf, _, _ = cpd.registration_cpd(points_vicon_pre, points_3dmd, 'rigid')

r_cab = tf.rot
s = tf.scale
t = tf.t

# combine pre-alignment and registration
r = np.matmul(r_cab, r_pre)

points_vicon_cab = tf.transform(points_vicon_pre)

In [ ]:
scene = pv.Plotter()
scene.add_points(points_3dmd, point_size=10, color='yellow', render_points_as_spheres=True)
scene.add_points(points_vicon_cab, point_size=10, color='green', render_points_as_spheres=True)
scene.add_mesh(mesh, texture=texture)
scene.camera_position = 'xy'
scene.show()

# Save calibration parameters

In [ ]:
from mesh4d import field

trans_cab = field.Trans_Rigid()
trans_cab.rot = r
trans_cab.scale = s
trans_cab.t = t

In [ ]:
from mesh4d import utils

utils.save_pkl_object(trans_cab, export_folder='output', export_name='vicon_to_3dmd')